In [3]:
import numpy as np
import pandas as pd
import json
import os
os.environ["OMP_NUM_THREADS"] = '1'


from langchain.prompts import PromptTemplate
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from umap import umap_ as UMAP
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt


In [11]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the API key
api_key = os.getenv("OPENAI_API_KEY")

# Verify the key is loaded
if api_key:
    print("API key loaded successfully!")
else:
    print("Failed to load API key. Check your .env file.")


API key loaded successfully!


In [20]:
from dotenv import load_dotenv
load_dotenv()

import os
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")

openai.api_key = openai_api_key
client = openai.Client()

chat_model_name = 'gpt-3.5-turbo'
embedding_model_name = 'sentence-transformers/all-mpnet-base-v2'

n_pick = 150  # the number of reviews picked for each game
s_root = r'C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis/'
s_db_json = 'review_db.json'
s_db_embed_json = 'review_db_embed.json'
s_db_table_json = 'review_db_table.json'
s_db_table_xlsx = 'review_db_table.xlsx'
s_db_table_pca_json = 'review_db_table_pca.json'
s_db_table_pca_xlsx = 'review_db_table_pca.xlsx'
s_kmeans_centers = 'kmeans_centers.json'
b_override = False

In [7]:
# df_app = pd.read_csv(s_root + 'reference_titles.csv')
# app_ids = [str(x) for x in df_app['app_id'].tolist()]

# we don't need this because I already have the data in a xlsx file

In [8]:
prompt_template_translation = PromptTemplate.from_template(
'''Please tell if the following game review is written in English.  Respond with "Yes." or "No."
Furthermore, please translate it into English if it is not, under "TRANSLATION:".


[h0]==================================================================[\h0]
REVIEW: 

"兄弟们，我把星空退款的钱拿来买这个了，我做的对吗"

IS ENGLISH: 

No.

TRANSLATION:

Brothers, I used the refund money from the stars to buy this. Did I do the right thing?


[h0]==================================================================[\h0]
REVIEW: 

"se puede acariciar el perro"

IS ENGLISH:

No.

TRANSLATION:

You can pet the dog.


[h0]==================================================================[\h0]
REVIEW: 

"My first D&D experience and I'm enjoying it a lot."

IS ENGLISH: 

Yes.


[h0]==================================================================[\h0]
REVIEW: 

"{review}"

IS ENGLISH: 

'''
)

prompt_template_topic = PromptTemplate.from_template(
'''Please list the most important topics and their respective original context in the review of a game in a json format with "Topic", "Category", "Context" arguments.  No more than 10 topics.
Topics should be game features.  A feature in the game should be a noun rather than a verb or an adjective.
Each topic should be categorized as a "fact" or a "request".
Topics should be translated to English if they are not.


[h0]==================================================================[\h0]
REVIEW: 

"My first D&D experience and I'm enjoying it a lot. However, I would prefer a little more guidance as the mechanics are quite overwhelming at first (especially the combat mechanics)."

TOPICS:

[
    {{
        "Topic": "D:D",
        "Category": "fact",
        "Context": "My first D&D experience and I'm enjoying it a lot.",
    }},
    {{
        "Topic": "combat mechanic",
        "Category": "fact",
        "Context": "the mechanics are quite overwhelming at first (especially the combat mechanics)",
    }}
]


[h0]==================================================================[\h0]
REVIEW: 

"I have only one point of criticism:
This game lacks a proper ending or epilog; something like The Witcher's Blood & Wine DLC. After adventuring hundreds of hours, after growing fond of and close to my companions, I got kicked so hard out of the game - it hurt!
In fact I immediately started a 2nd playthrough, because I wasn't yet ready to let go. Please Larian, give us an epilog or at least one last night at the camp with all my allies, companions and friends for the Definitive Edition."

TOPICS:

[
    {{
        "Topic": "epilogue",
        "Category": "fact",
        "Context": "This game lacks a proper ending or epilog",
    }},
    {{
        "Topic": "companion",
        "Category": "request",
        "Context": "Please Larian, give us an epilog or at least one last night at the camp with all my allies, companions and friends for the Definitive Edition",
    }}
]


[h0]==================================================================[\h0]
REVIEW: 

"{review}"

TOPICS:

'''
)

prompt_template_topic_view = PromptTemplate.from_template(
'''What's the sentiment of the review with regard to the topic?
Always answer with 'Positive' or 'Negative' or 'Inconclusive'

REVIEW: My first D&D experience and I'm enjoying it a lot.
TOPIC: D&D
SENTIMENT: Positive 

REVIEW: This game lacks a proper ending or epilog
TOPIC: epilogue
SENTIMENT: Negative

REVIEW: Posted: August 8
TOPIC: release date
SENTIMENT: Inconclusive 

REVIEW: {review}
TOPIC: {topic}
SENTIMENT: '''
)

In [14]:
# Read in the JSON file with survey results 

with open(s_root + 'Data/survey_results_clean.json', 'r') as f:
    db = json.load(f)

In [22]:
entry = db[0]

# Extract important information from the 2nd and 3rd keys
review_text = entry["Please tell us why you chose the rating above:"]
additional_feedback = entry["If you had a magic wand and you could change, add, or remove anything from the game, what would it be and why?"]

# Combine both into a single review input for the prompt
combined_review = f"{review_text} {additional_feedback}"

# Format the prompt for the LLM
prompt_topic = prompt_template_topic.format(review=combined_review)

# Make the OpenAI API call
response = client.chat.completions.create(
    model=chat_model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant expertised in game review analysis."},
        {"role": "user", "content": prompt_topic},
    ],
    max_tokens=1024,
)

# Print the response content
print(response.choices[0].message['content'])


TypeError: 'ChatCompletionMessage' object is not subscriptable

In [23]:
print(response)

ChatCompletion(id='chatcmpl-ASFqX7u8CiQuzLcgIj24wb0rHIZlT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='[\n    {\n        "Topic": "survival gameplay",\n        "Category": "fact",\n        "Context": "PROMETE SER UN JUEGO SURVIVAL DE ZOMBIES CON UNA JUGABILIDAD UNICA E INTERESANTE"\n    },\n    {\n        "Topic": "unique mechanics",\n        "Category": "fact",\n        "Context": "ME GUSTO EN VERDAD Y ESPERO CON ANSIAS Y CON ALTAS ESPECTATIVAS EL JUEGO ACABADO, COMO LO QUE ME DEJO EL DEMO"\n    },\n    {\n        "Topic": "resource management",\n        "Category": "fact",\n        "Context": "DEBIDO A QUE ES UN JUEGO QUE EN SI, ES DIFICIL POR SOBREVIVIR Y ESTAR PREOCUPADO POR RECURSOS, COSA QUE PONE INTERESANTE EL JUEGO"\n    }\n]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731297461, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=

In [24]:
 print(response.choices[0].message.content)

[
    {
        "Topic": "survival gameplay",
        "Category": "fact",
        "Context": "PROMETE SER UN JUEGO SURVIVAL DE ZOMBIES CON UNA JUGABILIDAD UNICA E INTERESANTE"
    },
    {
        "Topic": "unique mechanics",
        "Category": "fact",
        "Context": "ME GUSTO EN VERDAD Y ESPERO CON ANSIAS Y CON ALTAS ESPECTATIVAS EL JUEGO ACABADO, COMO LO QUE ME DEJO EL DEMO"
    },
    {
        "Topic": "resource management",
        "Category": "fact",
        "Context": "DEBIDO A QUE ES UN JUEGO QUE EN SI, ES DIFICIL POR SOBREVIVIR Y ESTAR PREOCUPADO POR RECURSOS, COSA QUE PONE INTERESANTE EL JUEGO"
    }
]


In [ ]:
prompt_topic = prompt_template_topic.format(review=tr[1]['review'])
response = openai.ChatCompletion.create(
    model=chat_model_name,
    temperature=0,
    messages=[
        {"role": "system", "content": "You are a helpful assistant expertised in game review analysis."},
        {"role": "user", "content": prompt_topic},
    ],
    max_tokens=1024,
)
print(response['choices'][0]['message']['content'])